# 中文连续词袋模型 (CBOW) 实现

本 Notebook 演示如何使用 Python 和 TensorFlow/Keras 构建和训练一个中文连续词袋模型 (CBOW)。

**目标:**

*   理解 CBOW 模型的基本原理。
*   使用 TensorFlow/Keras 实现 CBOW 模型。
*   准备中文训练数据。
*   使用分词工具处理中文文本。
*   训练模型并评估其性能。

## 1. 导入必要的库

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
import jieba  # 用于中文分词

## 2. 准备中文数据

我们使用一个简单的中文句子作为示例。实际应用中，需要使用更大的中文语料库。


In [ ]:
sentence = "自然语言处理是人工智能的一个重要分支。它涉及计算机对人类语言的理解和生成。"

# 1. 中文分词
seg_list = jieba.cut(sentence, cut_all=False)  # 使用精确模式分词
segmented_sentence = " ".join(seg_list)

print(f"分词结果: {segmented_sentence}")

# 2. 数据预处理
tokenizer = Tokenizer()
tokenizer.fit_on_texts([segmented_sentence])

word_index = tokenizer.word_index
vocabulary_size = len(word_index) + 1  # +1 for the padding token (if needed)

print(f"词汇表大小: {vocabulary_size}")
print(f"单词索引: {word_index}")

encoded_sentence = tokenizer.texts_to_sequences([segmented_sentence])[0]

## 3. 创建训练数据 (CBOW 上下文-目标词对)

In [3]:
def generate_cbow_pairs(corpus, window_size):
    context_length = 2 * window_size
    cbow_pairs = []
    for i in range(window_size, len(corpus) - window_size):
        context_words = [corpus[i - j] for j in range(window_size, 0, -1)]  # 前面的上下文词
        context_words += [corpus[i + j] for j in range(1, window_size + 1)]  # 后面的上下文词
        target_word = corpus[i]
        cbow_pairs.append((context_words, target_word))
    return cbow_pairs

window_size = 2
cbow_pairs = generate_cbow_pairs(encoded_sentence, window_size)

# 分离上下文词和目标词
context_words, target_words = zip(*cbow_pairs)

# 转换为 NumPy 数组
context_words = np.array(context_words)
target_words = np.array(target_words)

print(f"生成的 CBOW 词对数量: {len(cbow_pairs)}")
print(f"第一个词对: context={context_words[0]}, target={target_words[0]}")


## 4. 构建 CBOW 模型

In [4]:
# 4. 构建 CBOW 模型
embedding_dim = 50  # 词嵌入维度
context_length = 2 * window_size # 上下文长度

cbow_input = keras.Input(shape=(context_length,), dtype='int32') # 输入是上下文词序列
embedding = layers.Embedding(vocabulary_size, embedding_dim)(cbow_input)
average = layers.GlobalAveragePooling1D()(embedding) # 对词嵌入求平均
output = layers.Dense(vocabulary_size, activation='softmax')(average) # 使用 softmax 进行多分类

cbow = keras.Model(inputs=cbow_input, outputs=output)

cbow.summary()

## 5. 编译和训练模型

In [5]:
# 5. 编译模型
cbow.compile(optimizer='adam',  # Adam 优化器
              loss='sparse_categorical_crossentropy',  # 适用于多分类问题
              metrics=['accuracy'])

# 将目标词转换为 one-hot 编码
target_words_one_hot = tf.keras.utils.to_categorical(target_words, num_classes=vocabulary_size)

# 6. 训练模型
epochs = 100

cbow.fit(context_words, target_words_one_hot, epochs=epochs, batch_size=256, verbose=0)

print("训练完成!")

## 6. 评估 (可选) 和使用词嵌入

同样，由于数据集非常小，直接评估模型的质量意义不大。  通常，你需要在一个更大的数据集上训练模型，然后评估生成的词嵌入在其他任务上的表现。


In [ ]:
# 7. 获取词嵌入
embedding_layer = cbow.layers[1]  # Embedding 层现在是第二层
embeddings = embedding_layer.get_weights()[0]

print(f"词嵌入矩阵的形状: {embeddings.shape}")  # (vocabulary_size, embedding_dim)

# 获取 "计算机" 的词嵌入
word = "计算机"
word_id = word_index[word]
embedding_vector = embeddings[word_id]

print(f"'{word}' 的词嵌入: {embedding_vector}")

## 7. 完整代码（整合）

```python
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import skipgrams
import jieba  # 用于中文分词

sentence = "我 爱 北京 天安门。 天安门 上 太阳 升。"

# 1. 中文分词
seg_list = jieba.cut(sentence, cut_all=False)  # 使用精确模式分词
segmented_sentence = " ".join(seg_list)

print(f"分词结果: {segmented_sentence}")

# 2. 数据预处理
tokenizer = Tokenizer()
tokenizer.fit_on_texts([segmented_sentence])

word_index = tokenizer.word_index
vocabulary_size = len(word_index) + 1  # +1 for the padding token (if needed)

print(f"词汇表大小: {vocabulary_size}")
print(f"单词索引: {word_index}")

encoded_sentence = tokenizer.texts_to_sequences([segmented_sentence])[0]

def generate_cbow_pairs(corpus, window_size):
    context_length = 2 * window_size
    cbow_pairs = []
    for i in range(window_size, len(corpus) - window_size):
        context_words = [corpus[i - j] for j in range(window_size, 0, -1)]  # 前面的上下文词
        context_words += [corpus[i + j] for j in range(1, window_size + 1)]  # 后面的上下文词
        target_word = corpus[i]
        cbow_pairs.append((context_words, target_word))
    return cbow_pairs

window_size = 2
cbow_pairs = generate_cbow_pairs(encoded_sentence, window_size)

# 分离上下文词和目标词
context_words, target_words = zip(*cbow_pairs)

# 转换为 NumPy 数组
context_words = np.array(context_words)
target_words = np.array(target_words)

print(f"生成的 CBOW 词对数量: {len(cbow_pairs)}")
print(f"第一个词对: context={context_words[0]}, target={target_words[0]}")

# 4. 构建 CBOW 模型
embedding_dim = 50  # 词嵌入维度
context_length = 2 * window_size # 上下文长度

cbow_input = keras.Input(shape=(context_length,), dtype='int32') # 输入是上下文词序列
embedding = layers.Embedding(vocabulary_size, embedding_dim)(cbow_input)
average = layers.GlobalAveragePooling1D()(embedding) # 对词嵌入求平均
output = layers.Dense(vocabulary_size, activation='softmax')(average) # 使用 softmax 进行多分类

cbow = keras.Model(inputs=cbow_input, outputs=output)

cbow.summary()

# 5. 编译模型
cbow.compile(optimizer='adam',  # Adam 优化器
              loss='sparse_categorical_crossentropy',  # 适用于多分类问题
              metrics=['accuracy'])

# 将目标词转换为 one-hot 编码
target_words_one_hot = tf.keras.utils.to_categorical(target_words, num_classes=vocabulary_size)

# 6. 训练模型
epochs = 100

cbow.fit(context_words, target_words_one_hot, epochs=epochs, batch_size=256, verbose=0)

print("训练完成!")

# 7. 获取词嵌入
embedding_layer = cbow.layers[1]  # Embedding 层现在是第二层
embeddings = embedding_layer.get_weights()[0]

print(f"词嵌入矩阵的形状: {embeddings.shape}")  # (vocabulary_size, embedding_dim)

# 获取 "北京" 的词嵌入
word = "北京"
word_id = word_index[word]
embedding_vector = embeddings[word_id]

print(f"'{word}' 的词嵌入: {embedding_vector}")
```

## 总结

本 Notebook 演示了如何使用 TensorFlow/Keras 构建和训练一个中文 CBOW 模型。 关键的修改包括：

*   **数据准备：** 生成 CBOW 上下文-目标词对，并对目标词进行 one-hot 编码。
*   **模型结构：** 使用 `keras.Input` 定义输入层，接受多个上下文词。 使用 `GlobalAveragePooling1D` 对上下文词的词向量求平均。 使用 `softmax` 激活函数和 `categorical_crossentropy` 损失函数。
*   **训练：** 将上下文词作为输入，one-hot 编码的目标词作为输出进行训练。